In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task5
from labels_to_ids import task5_labels_to_ids
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
        tr_f1score += tmp_tr_f1score
    

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../2022.07.07_task5/'

    train_data = read_task5(dataset_location , split = 'train')
    dev_data = read_task5(dataset_location , split = 'dev')
    #test_data = read_task5(dataset_location , split = 'dev')#load test set
    labels_to_ids = task5_labels_to_ids
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['bert-base-multilingual-uncased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.4751075804233551
DEV ACC: 0.8275095532039977
DEV Precision: 0.6516617421379327
DEV Recall: 0.8734819853867473
DEV F1Score: 0.7274434611969623
BEST ACCURACY -->  DEV: 0.82751
BEST PRECISION -->  DEV: 0.65166
BEST RECALL -->  DEV: 0.87348
BEST F1SCORE -->  DEV: 0.72744
TIME PER EPOCH: 6.384122586250305

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIH

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.4512663781642914
DEV ACC: 0.822145061728395
DEV Precision: 0.6358817989770372
DEV Recall: 0.8796605510891227
DEV F1Score: 0.7148975360083054
BEST ACCURACY -->  DEV: 0.82215
BEST PRECISION -->  DEV: 0.63588
BEST RECALL -->  DEV: 0.87966
BEST F1SCORE -->  DEV: 0.7149
TIME PER EPOCH: 6.6842606584231055

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.40996238589286804

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.3797430694103241
DEV ACC: 0.833370076425632
DEV Precision: 0.670828818447866
DEV Recall: 0.8387422982661077
DEV F1Score: 0.7290824824012009
BEST ACCURACY -->  DEV: 0.83337
BEST PRECISION -->  DEV: 0.67083
BEST RECALL -->  DEV: 0.83874
BEST F1SCORE -->  DEV: 0.72908
TIME PER EPOCH: 6.601263535022736

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.37207671999931335
DEV ACC: 0.8270502645502645
DEV Precision: 0.6257899067422878
DEV Recall: 0.9012786596119927
DEV F1Score: 0.721664299774636
BEST ACCURACY -->  DEV: 0.82705
BEST PRECISION -->  DEV: 0.62579
BEST RECALL -->  DEV: 0.90128
BEST F1SCORE -->  DEV: 0.72166
TIME PER EPOCH: 6.592431465784709

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIH

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.5256771445274353
DEV ACC: 0.8139329805996472
DEV Precision: 0.626622231384136
DEV Recall: 0.8723670949861424
DEV F1Score: 0.7147476334008841
BEST ACCURACY -->  DEV: 0.81393
BEST PRECISION -->  DEV: 0.62662
BEST RECALL -->  DEV: 0.87237
BEST F1SCORE -->  DEV: 0.71475
TIME PER EPOCH: 6.673238436381022

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHE

In [6]:
print(best_dev_acc)

0.8498309817754263


In [7]:
print(best_epoch)

4


In [8]:
print(best_dev_precision)

0.6802938683891065


In [9]:
print(best_dev_recall)

0.8961817723722485


In [16]:
print(best_dev_f1score)

0.7472336999617473


In [11]:
print(overall_list_dev_acc)

[[0.8275095532039977, 0.851410934744268, 0.8413065843621399, 0.841086125808348, 0.8455504115226337, 0.8406819517930629, 0.8273258377425045, 0.8433825690770136, 0.8323780129335685, 0.8165968547912993], [0.822145061728395, 0.8400022045855379, 0.8457708700764257, 0.8592555849500294, 0.8336823927101705, 0.8436030276308054, 0.845679012345679, 0.8208774250440917, 0.8148883009994121, 0.8174970605526162], [0.833370076425632, 0.8405349794238683, 0.8515027924750147, 0.8409942680776014, 0.8460831863609641, 0.8473508230452675, 0.8278218694885362, 0.8363462669018225, 0.8188932980599647, 0.8244415049970606], [0.8270502645502645, 0.8580798059964727, 0.852935773074662, 0.8404982363315696, 0.8455504115226337, 0.843290711346267, 0.8450911228689006, 0.8453666960611406, 0.8344539976484422, 0.8250293944738389], [0.8139329805996472, 0.8433825690770136, 0.848342886537331, 0.8498309817754263, 0.8478468547912993, 0.8331863609641388, 0.8364381246325692, 0.8042879188712522, 0.8126286008230453, 0.8284097589653145

In [12]:
print(overall_list_dev_precision)

[[0.6516617421379327, 0.6956383563526418, 0.6606742023408692, 0.671081563938707, 0.6790862136100229, 0.6605556700794798, 0.6459624590576973, 0.6719118623880527, 0.705750818846057, 0.6359914336104809], [0.6358817989770372, 0.7208679768203576, 0.6519732472113425, 0.7018421613659711, 0.6947004054146912, 0.6762208250303492, 0.6851253244110388, 0.602355757117662, 0.6703514739229024, 0.7081632653061224], [0.670828818447866, 0.7016124968505921, 0.66465778489588, 0.6391459950983762, 0.6561451775737489, 0.6820512380036191, 0.686266863647816, 0.6712977058215154, 0.6499244142101285, 0.624505829267734], [0.6257899067422878, 0.6751768073196645, 0.6824818479580385, 0.6338114707162327, 0.6372036693465265, 0.6967082740892265, 0.7032473258663736, 0.6584965299251015, 0.6607016880826404, 0.6625661375661377], [0.626622231384136, 0.6434429327286472, 0.6604744461887317, 0.6802938683891065, 0.6661429840001268, 0.6627350603541081, 0.6730462218557458, 0.6167222107698298, 0.6558390022675737, 0.6333144809335286]

In [13]:
print(overall_list_dev_recall)

[[0.8734819853867473, 0.8640778533635675, 0.8104159508921416, 0.8672335600907026, 0.8608150667674477, 0.9059712773998488, 0.6630070546737213, 0.8153233010375868, 0.6046914725486155, 0.6784729379967475], [0.8796605510891227, 0.7460443436633912, 0.8928124785267642, 0.8480284706475183, 0.7162359245692579, 0.786183305230924, 0.790432098765432, 0.6083762798048513, 0.6295695045695043, 0.5209029066171924], [0.8387422982661077, 0.7084278155706725, 0.9025447215923406, 0.8531413912366292, 0.808636072921787, 0.8079737282118233, 0.7772343617581713, 0.7973596509310794, 0.6555177626606199, 0.7593067866877389], [0.9012786596119927, 0.8665658856135046, 0.8590073066263543, 0.9378936759889138, 0.8889444329920518, 0.7967750062988158, 0.7672650541698163, 0.8161238232666802, 0.7465041572184429, 0.7305303602922651], [0.8723670949861424, 0.8690413202317965, 0.8791131267321741, 0.8086093359902883, 0.8961817723722485, 0.8091945532421722, 0.7056740878169451, 0.696214411690602, 0.5849561373370898, 0.759063995968

In [14]:
print(overall_list_dev_f1score)

[[0.7274434611969623, 0.759513970803223, 0.7113945422413199, 0.7447870762149104, 0.7392940470671556, 0.7403282136475413, 0.6345340296726115, 0.7129163519262804, 0.6257453667817812, 0.6366347264106367], [0.7148975360083054, 0.7061160387490916, 0.7400128672491162, 0.7457081560144522, 0.6881891062563328, 0.7113497628498363, 0.7153888229592195, 0.586245417754337, 0.6238945461634534, 0.5633336588223806], [0.7290824824012009, 0.6880682566514794, 0.7491992931490012, 0.71241527827478, 0.7044405527856801, 0.7231291340129611, 0.7039511246093878, 0.7066800739843639, 0.6253114558436689, 0.6670851742870023], [0.721664299774636, 0.7457879491536201, 0.7441165588224411, 0.7393903235604671, 0.7273733640459275, 0.7222069873219809, 0.7093788561649389, 0.7104299345992131, 0.6810003954961937, 0.6760676583505714], [0.7147476334008841, 0.7254809955953799, 0.7324540784293104, 0.7265348018185996, 0.7472336999617473, 0.7100555370937207, 0.6715173611532151, 0.6387131454723709, 0.5966439619100683, 0.6666867781750

In [15]:
#Best model is model 0